In [1]:
%matplotlib inline
import utils; reload(utils)
from utils import *

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
!pwd

/home/ubuntu/courses/deeplearning1/lesson2


In [61]:
path = '../data/fishies'
train_path = path + '/train'
valid_path = path + '/valid'
test_path = path + '/test_stg1'
results_path = path + '/results'
batch_size=64

In [62]:
train_batches = get_batches(train_path, batch_size=64)
valid_batches = get_batches(valid_path, batch_size=64)
test_batches = get_batches(test_path, batch_size=64)

Found 3064 images belonging to 8 classes.
Found 713 images belonging to 8 classes.
Found 1000 images belonging to 1 classes.


In [63]:
train_batches.class_indices

{'ALB': 0,
 'BET': 1,
 'DOL': 2,
 'LAG': 3,
 'NoF': 4,
 'OTHER': 5,
 'SHARK': 6,
 'YFT': 7}

In [64]:
from vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model

In [65]:
# Setup so the last layer only has 8 classes, one for each of our classes
vgg.finetune(train_batches)

In [66]:
vgg.fit(train_batches, valid_batches, nb_epoch=1)

Epoch 1/1
3064/3064 [==============================] - 104s - loss: 2.1329 - acc: 0.4677 - val_loss: 1.0387 - val_acc: 0.6914


In [68]:
vgg.fit(train_batches, valid_batches, nb_epoch=1)

Epoch 1/1
3064/3064 [==============================] - 104s - loss: 1.3493 - acc: 0.6012 - val_loss: 0.7949 - val_acc: 0.7770


In [69]:
vgg.fit(train_batches, valid_batches, nb_epoch=1)

Epoch 1/1
3064/3064 [==============================] - 104s - loss: 1.0574 - acc: 0.6717 - val_loss: 0.5996 - val_acc: 0.8331


In [70]:
vgg.fit(train_batches, valid_batches, nb_epoch=1)

Epoch 1/1
3064/3064 [==============================] - 104s - loss: 0.9277 - acc: 0.7190 - val_loss: 0.5941 - val_acc: 0.8289


In [75]:
_, valid_preds = vgg.test(valid_path)

Found 713 images belonging to 8 classes.


In [84]:
_, train_preds = vgg.test(train_path)

Found 3064 images belonging to 8 classes.


In [113]:
def onehot(batches):
    num_classes = len(batches.class_indices)
    return np.equal(batches.classes, np.matrix(np.arange(num_classes)).T).T.astype(int)
def multiclass_logloss_rowscaled(batches, preds):
    preds = np.clip(preds, 0.0001, 0.9999)
    print preds[:2]
    sums = np.matrix(np.sum(preds, axis=1)).T
    print sums[:2]
    preds = np.divide(preds, sums)
    print preds[:2]
    oh = onehot(batches)
    return (-1.0/batches.N) * np.sum(np.multiply(oh, np.log(preds)))

In [114]:
multiclass_logloss_rowscaled(valid_batches, valid_preds)

[[  9.9170e-01   1.3510e-03   1.0000e-04   1.0000e-04   3.1356e-03   1.0000e-04   1.9647e-03
    1.8123e-03]
 [  7.6093e-01   1.0301e-04   7.8499e-03   1.0000e-04   2.1446e-03   4.9946e-03   3.9654e-02
    1.8428e-01]]
[[ 1.0003]
 [ 1.0001]]
[[  9.9144e-01   1.3506e-03   9.9974e-05   9.9974e-05   3.1348e-03   9.9974e-05   1.9642e-03
    1.8118e-03]
 [  7.6089e-01   1.0300e-04   7.8495e-03   9.9995e-05   2.1444e-03   4.9943e-03   3.9652e-02
    1.8427e-01]]


0.61586382749410762

In [112]:
multiclass_logloss_rowscaled(train_batches, train_preds)

0.39243241152919128

In [105]:
np.sum(np.matrix([[1,2,3],[4,5,6]]), axis=1)

matrix([[ 6],
        [15]])

In [71]:
test_batches, preds = vgg.test(test_path)

Found 1000 images belonging to 1 classes.


In [94]:
train_batches.class_indices

{'ALB': 0,
 'BET': 1,
 'DOL': 2,
 'LAG': 3,
 'NoF': 4,
 'OTHER': 5,
 'SHARK': 6,
 'YFT': 7}

In [116]:
for i in train_batches.class_indices:
    print i

SHARK
DOL
NoF
LAG
ALB
YFT
OTHER
BET


In [117]:
# Generate CSV
sorted_indices = sorted(train_batches.class_indices)
header = ','.join(['image'] + [i for i in sorted(train_batches.class_indices)])
with open(results_path + '/results.csv', 'w') as f:
    f.write(header + '\n')
    for filename, prow in zip(test_batches.filenames, preds):
        f.write(filename.replace('UNKNOWN/','') + ',' + ','.join(map(str, prow)) + '\n')